In [ ]:
import os.path
from time import perf_counter

import numpy as np
import scipy.linalg as la
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc4
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.art3d as art3d

import bin_model as bm

# Bin Model Example and Animation

In [ ]:
KERNEL_FILE_NAME = "kernels/Hall_ScottChen_kernel_nb336.nc"
EXPERIMENT_FILE_NAME = \
    "convergence_experiments/mass_convergence_experiment_nb336.nc"

In [ ]:
with nc4.Dataset(KERNEL_FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    _, kernel, _, ktens = netcdf_file.read_cgk()
with nc4.Dataset(EXPERIMENT_FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp = netcdf_file.read_full_experiment([ktens])

In [ ]:
nt = exp.num_time_steps
nb = exp.mass_grid.num_bins
bb = exp.mass_grid.bin_bounds
times = exp.times
dsds = np.zeros((nt, nb))
for i in range(nt):
    dsds[i,:] = exp.states[i].dsd()
plt.pcolor(bb, times, np.log10(np.maximum(dsds[1:,:], 1.e-9)), cmap='jet')
plt.colorbar()

In [ ]:
const = exp.constants
grid = exp.mass_grid
ymin = 0.
ymax = 3.
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False,
                     xlim=(2. + grid.lx_min/(3.*np.log(10)),
                           2. + grid.lx_max/(3.*np.log(10))),
                     ylim=(ymin, ymax))
ax.set_xlabel("$log_{10}(D)$ ($D$ in microns)")
ax.set_ylabel("$dm/dlog_{10}(D)$ (g/kg)")
ax.grid()

plot_lds = 2. + grid.bin_bounds[:-1] / (3.*np.log(10))
line, = ax.plot(plot_lds, exp.states[0].dsd(),
                'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
plt.vlines(np.log10(const.rain_d)+6., ymin, ymax, 'k')

mass_convert = 1.e3 * (3.*np.log(10))
frame_stride = 1
def animate(i):
    ti = frame_stride * i
    thisy = mass_convert * exp.states[ti].dsd()
    line.set_data(plot_lds, thisy)
    time_text.set_text(time_template % exp.times[ti])
    return line, time_text

num_steps = exp.num_time_steps
num_frames = num_steps // frame_stride
ani = animation.FuncAnimation(
    fig, animate, num_frames,
    interval=6000. / num_frames,
    blit=True)
ani.save("mass_evolution.gif")
plt.show()

# Plots of error over time

In [ ]:
KERNEL_FILE_NAME = "kernels/Hall_ScottChen_kernel_nb168.nc"
with nc4.Dataset(KERNEL_FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    const, _, grid, ktens = netcdf_file.read_cgk()

## 2-category

In [ ]:
C0_3R0_3_80MICRON_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3r0-3_80micron.nc")
with nc4.Dataset(C0_3R0_3_80MICRON_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3r0_3_80micron = netcdf_file.read_full_experiment([ktens])
C0_3R0_3_50MICRON_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3r0-3_50micron.nc")
with nc4.Dataset(C0_3R0_3_50MICRON_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3r0_3_50micron = netcdf_file.read_full_experiment([ktens])
C0_3_6R0_3_80MICRON_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3-6r0-3_80micron.nc")
with nc4.Dataset(C0_3_6R0_3_80MICRON_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3_6r0_3_80micron = netcdf_file.read_full_experiment([ktens])
C0_3_6R0_3_50MICRON_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3-6r0-3_50micron.nc")
with nc4.Dataset(C0_3_6R0_3_50MICRON_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3_6r0_3_50micron = netcdf_file.read_full_experiment([ktens])
C0_3_6R0_3_6_80MICRON_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3-6r0-3-6_80micron.nc")
with nc4.Dataset(C0_3_6R0_3_6_80MICRON_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3_6r0_3_6_80micron = netcdf_file.read_full_experiment([ktens])
C0_3_6R0_3_6_50MICRON_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3-6r0-3-6_50micron.nc")
with nc4.Dataset(C0_3_6R0_3_6_50MICRON_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3_6r0_3_6_50micron = netcdf_file.read_full_experiment([ktens])

In [ ]:
t_eval = exp_c0_3r0_3_80micron.times

In [ ]:
wvs = np.zeros((3, grid.num_bins))
wvs[0,:] = grid.moment_weight_vector(0)
wvs[1,:] = grid.moment_weight_vector(3, cloud_only=True)
wvs[2,:] = grid.moment_weight_vector(6)
wvs_50micron = np.zeros((3, exp_c0_3r0_3_50micron.mass_grid.num_bins))
wvs_50micron[0,:] = exp_c0_3r0_3_50micron.mass_grid.moment_weight_vector(0)
wvs_50micron[1,:] = exp_c0_3r0_3_50micron.mass_grid.moment_weight_vector(
    3, cloud_only=True)
wvs_50micron[2,:] = exp_c0_3r0_3_50micron.mass_grid.moment_weight_vector(6)

In [ ]:
moms_c0_3r0_3_80micron, mom_covs_c0_3r0_3_80micron = \
    exp_c0_3r0_3_80micron.get_moments_and_covariances(wvs)
moms_c0_3r0_3_50micron, mom_covs_c0_3r0_3_50micron = \
    exp_c0_3r0_3_50micron.get_moments_and_covariances(wvs_50micron)
moms_c0_3_6r0_3_80micron, mom_covs_c0_3_6r0_3_80micron = \
    exp_c0_3_6r0_3_80micron.get_moments_and_covariances(wvs)
moms_c0_3_6r0_3_50micron, mom_covs_c0_3_6r0_3_50micron = \
    exp_c0_3_6r0_3_50micron.get_moments_and_covariances(wvs_50micron)
moms_c0_3_6r0_3_6_80micron, mom_covs_c0_3_6r0_3_6_80micron = \
    exp_c0_3_6r0_3_6_80micron.get_moments_and_covariances(wvs)
moms_c0_3_6r0_3_6_50micron, mom_covs_c0_3_6r0_3_6_50micron = \
    exp_c0_3_6r0_3_6_50micron.get_moments_and_covariances(wvs_50micron)

In [ ]:
U0_3_6_9_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_u0-3-6-9_no3err.nc")
with nc4.Dataset(U0_3_6_9_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_u0_3_6_9 = netcdf_file.read_full_experiment([ktens])
U0_3_4_5_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_u0-3-4-5_no3err.nc")
with nc4.Dataset(U0_3_4_5_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_u0_3_4_5 = netcdf_file.read_full_experiment([ktens])
U0_3_6_9_12_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_u0-3-6-9-12.nc")
with nc4.Dataset(U0_3_6_9_12_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_u0_3_6_9_12 = netcdf_file.read_full_experiment([ktens])
U0_1_2_3_4_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_u0-1-2-3-4.nc")
with nc4.Dataset(U0_1_2_3_4_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_u0_1_2_3_4 = netcdf_file.read_full_experiment([ktens])

In [ ]:
moms_u0_3_6_9, mom_covs_u0_3_6_9 = \
    exp_u0_3_6_9.get_moments_and_covariances(wvs)
moms_u0_3_4_5, mom_covs_u0_3_4_5 = \
    exp_u0_3_4_5.get_moments_and_covariances(wvs)
moms_u0_3_6_9_12, mom_covs_u0_3_6_9_12 = \
    exp_u0_3_6_9_12.get_moments_and_covariances(wvs)
moms_u0_1_2_3_4, mom_covs_u0_1_2_3_4 = \
    exp_u0_1_2_3_4.get_moments_and_covariances(wvs)
moms_u0_3_6_9_50micron, mom_covs_u0_3_6_9_50micron = \
    exp_u0_3_6_9.get_moments_and_covariances(wvs_50micron)
moms_u0_3_4_5_50micron, mom_covs_u0_3_4_5_50micron = \
    exp_u0_3_4_5.get_moments_and_covariances(wvs_50micron)

In [ ]:
fig = plt.figure(figsize=(7, 4))
# rho_air is 1.2, 1.e-6 converts from m^-3 to cm^-3
m0_scale = 1.2e-6 * const.mass_conc_scale / const.std_mass
plt.plot(t_eval, m0_scale * moms_c0_3r0_3_80micron[:,0],
         'k', label='Total M0 ($cm^{-3}$)')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3r0_3_80micron[:,0,0]),
         color='b', label='C0-3R0-3, $80\mu m$')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3r0_3_50micron[:,0,0]),
         color='b', linestyle='--', label='C0-3R0-3, $50\mu m$')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3_6r0_3_80micron[:,0,0]),
         color='orange', label='C0-3-6R0-3, $80\mu m$')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3_6r0_3_50micron[:,0,0]),
         color='orange', linestyle='--', label='C0-3-6R0-3, $50\mu m$')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3_6r0_3_6_80micron[:,0,0]),
         color='g', label='C0-3-6R0-3-6, $80\mu m$')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3_6r0_3_6_50micron[:,0,0]),
         color='g', linestyle='--', label='C0-3-6R0-3-6, $50\mu m$')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_u0_3_6_9[:,0,0]),
         color='r', label='U0-3-6-9')
plt.xlim(0., 3600.)
plt.ylim(0., 275.)
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
# 1.e3 converts kg/kg to g/kg
mass_scale = 1.e3 * const.mass_conc_scale
plt.plot(t_eval, mass_scale * moms_c0_3r0_3_80micron[:,1],
         'k', label='Cloud mass, $80\mu m$')
plt.plot(t_eval, mass_scale * moms_c0_3r0_3_50micron[:,1],
         'k--', label='Cloud mass, $50\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3r0_3_80micron[:,1,1]),
         color='b', label='C0-3R0-3, $80\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3r0_3_50micron[:,1,1]),
         color='b', linestyle='--', label='C0-3R0-3, $50\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3_6r0_3_80micron[:,1,1]),
         color='orange', label='C0-3-6R0-3, $80\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3_6r0_3_50micron[:,1,1]),
         color='orange', linestyle='--', label='C0-3-6R0-3, $50\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3_6r0_3_6_80micron[:,1,1]),
         color='g', label='C0-3-6R0-3-6, $80\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3_6r0_3_6_50micron[:,1,1]),
         color='g', linestyle='--', label='C0-3-6R0-3-6, $50\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_3_6_9[:,1,1]),
         color='r', label='U0-3-6-9, $80\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_3_6_9_50micron[:,1,1]),
         color='r', linestyle='--', label='U0-3-6-9, $50\mu m$')
plt.xlim(0., 3600.)
plt.ylim(0., 1.4)
plt.xlabel('Time (s)')
plt.ylabel('Cloud mass (g/kg)')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
# 1.e3 to get mm^6/cm^3
m6_scale = (1.e3 * const.std_diameter)**6 * m0_scale
plt.plot(t_eval, m6_scale * moms_c0_3r0_3_80micron[:,2],
         'k', label='Total $M_6$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3r0_3_80micron[:,2,2]),
         color='b', label='C0-3R0-3, $80\mu m$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3r0_3_50micron[:,2,2]),
         color='b', linestyle='--', label='C0-3R0-3, $50\mu m$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3_6r0_3_80micron[:,2,2]),
         color='orange', label='C0-3-6R0-3, $80\mu m$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3_6r0_3_50micron[:,2,2]),
         color='orange', linestyle='--', label='C0-3-6R0-3, $50\mu m$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3_6r0_3_6_80micron[:,2,2]),
         color='g', label='C0-3-6R0-3-6, $80\mu m$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3_6r0_3_6_50micron[:,2,2]),
         color='g', linestyle='--', label='C0-3-6R0-3-6, $50\mu m$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_u0_3_6_9[:,2,2]),
         color='r', label='U0-3-6-9')
plt.xlim(0., 3600.)
plt.ylim(0., 0.25)
plt.xlabel('Time (s)')
plt.ylabel('$M_6$ ($mm^6cm^{-3}$)')
plt.legend()

## 1-category

In [ ]:
fig = plt.figure(figsize=(7, 4))
# rho_air is 1.2, 1.e-6 converts from m^-3 to cm^-3
m0_scale = 1.2e-6 * const.mass_conc_scale / const.std_mass
plt.plot(t_eval, m0_scale * moms_c0_3r0_3_80micron[:,0],
         'k', label='Total M0 ($cm^{-3}$)')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_u0_3_6_9[:,0,0]),
         color='r', label='U0-3-6-9')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_u0_3_4_5[:,0,0]),
         color='purple', label='U0-3-4-5')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_u0_3_6_9_12[:,0,0]),
         color='brown', label='U0-3-6-9-12')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_u0_1_2_3_4[:,0,0]),
         color='pink', label='U0-1-2-3-4')
plt.xlim(0., 3600.)
plt.ylim(0., 275.)
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
# 1.e3 converts kg/kg to g/kg
mass_scale = 1.e3 * const.mass_conc_scale
plt.plot(t_eval, mass_scale * moms_c0_3r0_3_80micron[:,1],
         'k', label='Cloud mass')
#plt.plot(t_eval, mass_scale * moms_c0_3r0_3_50micron[:,1],
#         'k', linestyle='--', label='Cloud mass')
#plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3r0_3_50micron[:,1,1]),
#         color='b', label='C0-3R0-3')
#plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3_6r0_3_80micron[:,1,1]),
#         color='orange', label='C0-3-6R0-3')
#plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3_6r0_3_6_80micron[:,1,1]),
#         color='g', label='C0-3-6R0-3-6')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_3_6_9[:,1,1]),
         color='r', label='U0-3-6-9')
#plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_3_6_9_50micron[:,1,1]),
#         color='r', linestyle='--', label='U0-3-6-9')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_3_4_5[:,1,1]),
         color='purple', label='U0-3-4-5')
#plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_3_4_5_50micron[:,1,1]),
#         color='purple', linestyle='--', label='U0-3-4-5')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_3_6_9_12[:,1,1]),
         color='brown', label='U0-3-6-9-12')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_u0_1_2_3_4[:,1,1]),
         color='pink', label='U0-1-2-3-4')
plt.xlim(0., 3600.)
plt.ylim(0., 1.4)
plt.xlabel('Time (s)')
plt.ylabel('Cloud mass (g/kg)')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
# 1.e3 to get mm^6/cm^3
m6_scale = (1.e3 * const.std_diameter)**6 * m0_scale
plt.plot(t_eval, m6_scale * moms_c0_3r0_3_80micron[:,2],
         'k', label='Total $M_6$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_u0_3_6_9[:,2,2]),
         color='r', label='U0-3-6-9')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_u0_3_4_5[:,2,2]),
         color='purple', label='U0-3-4-5')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_u0_3_6_9_12[:,2,2]),
         color='brown', label='U0-3-6-9-12')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_u0_1_2_3_4[:,2,2]),
         color='pink', label='U0-1-2-3-4')
plt.xlim(0., 3600.)
plt.ylim(0., 0.25)
plt.xlabel('Time (s)')
plt.ylabel('$M_6$ ($mm^6cm^{-3}$)')
plt.legend()

## 1-category zeta uncertainty

In [ ]:
fig = plt.figure(figsize=(7, 4))
plt.plot(t_eval, np.sqrt(exp_u0_3_6_9.zeta_cov[:,0,0]),
         color='r', label='U0-3-6-9')
plt.plot(t_eval, np.sqrt(exp_u0_3_4_5.zeta_cov[:,0,0]),
         color='brown', label='U0-3-4-5')
plt.ylim(0., 50.*np.sqrt(exp_u0_3_6_9.zeta_cov[1,0,0]))
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
plt.plot(t_eval, np.sqrt(exp_u0_3_6_9.zeta_cov[:,1,1]),
         color='r', label='U0-3-6-9')
plt.plot(t_eval, np.sqrt(exp_u0_3_4_5.zeta_cov[:,1,1]),
         color='brown', label='U0-3-4-5')
plt.ylim(0., 50.*np.sqrt(exp_u0_3_6_9.zeta_cov[1,1,1]))
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
plt.plot(t_eval, np.sqrt(exp_u0_3_6_9.zeta_cov[:,2,2]),
         color='r', label='U0-3-6-9')
plt.plot(t_eval, np.sqrt(exp_u0_3_4_5.zeta_cov[:,2,2]),
         color='brown', label='U0-3-4-5')
plt.ylim(0., 50.*np.sqrt(exp_u0_3_6_9.zeta_cov[1,2,2]))
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
plt.plot(t_eval, mass_scale * np.sqrt(exp_u0_3_6_9.zeta_cov[:,3,3]),
         color='r', label='U0-3-6-9')
plt.plot(t_eval, mass_scale * np.sqrt(exp_u0_3_4_5.zeta_cov[:,3,3]),
         color='brown', label='U0-3-4-5')
plt.ylim(0., 50.*np.sqrt(exp_u0_3_6_9.zeta_cov[1,3,3]))
plt.legend()

## 1-category moment evolution

In [ ]:
wvs_test = np.zeros((6, grid.num_bins))
wvs_test[0,:] = grid.moment_weight_vector(0)
wvs_test[1,:] = grid.moment_weight_vector(3)
wvs_test[2,:] = grid.moment_weight_vector(4)
wvs_test[3,:] = grid.moment_weight_vector(5)
wvs_test[4,:] = grid.moment_weight_vector(6)
wvs_test[5,:] = grid.moment_weight_vector(9)

In [ ]:
moms_test, mom_covs_test = \
    exp_u0_3_4_5.get_moments_and_covariances(wvs_test)

In [ ]:
plt.plot(t_eval, np.log(moms_test[:,0]))

In [ ]:
plt.plot(t_eval, np.log(moms_test[:,1]))

In [ ]:
plt.plot(t_eval, np.log(moms_test[:,2]))

In [ ]:
plt.plot(t_eval, np.log(moms_test[:,3]))

In [ ]:
plt.plot(t_eval, np.log(moms_test[:,4]))

In [ ]:
plt.plot(t_eval, np.log(moms_test[:,5]))

## Two-category "hard" case

In [ ]:
C0_3R0_3_80MICRON_HARDCASE_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3r0-3_80micron_hardcase.nc")
with nc4.Dataset(C0_3R0_3_80MICRON_HARDCASE_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3r0_3_80micron_hardcase = netcdf_file.read_full_experiment([ktens])
C0_3_6R0_3_80MICRON_HARDCASE_FILE = os.path.join(
    "sensitivity_experiments", "experiment_aguset_c0-3-6r0-3_80micron_hardcase.nc")
with nc4.Dataset(C0_3_6R0_3_80MICRON_HARDCASE_FILE, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_c0_3_6r0_3_80micron_hardcase = netcdf_file.read_full_experiment([ktens])

In [ ]:
moms_c0_3r0_3_80micron_hardcase, mom_covs_c0_3r0_3_80micron_hardcase = \
    exp_c0_3r0_3_80micron_hardcase.get_moments_and_covariances(wvs)
moms_c0_3_6r0_3_80micron_hardcase, mom_covs_c0_3_6r0_3_80micron_hardcase = \
    exp_c0_3_6r0_3_80micron_hardcase.get_moments_and_covariances(wvs)

In [ ]:
fig = plt.figure(figsize=(7, 4))
# rho_air is 1.2, 1.e-6 converts from m^-3 to cm^-3
m0_scale = 1.2e-6 * const.mass_conc_scale / const.std_mass
plt.plot(t_eval, m0_scale * moms_c0_3r0_3_80micron_hardcase[:,0],
         'k', label='Total M0 ($cm^{-3}$)')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3r0_3_80micron_hardcase[:,0,0]),
         color='b', label='C0-3R0-3, $80\mu m$')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_c0_3_6r0_3_80micron_hardcase[:,0,0]),
         color='orange', label='C0-3-6R0-3, $80\mu m$')
plt.xlim(0., 3600.)
plt.ylim(0., 880.)
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
# 1.e3 converts kg/kg to g/kg
mass_scale = 1.e3 * const.mass_conc_scale
plt.plot(t_eval, mass_scale * moms_c0_3r0_3_80micron_hardcase[:,1],
         'k', label='Cloud mass, $80\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3r0_3_80micron_hardcase[:,1,1]),
         color='b', label='C0-3R0-3, $80\mu m$')
plt.plot(t_eval, mass_scale * np.sqrt(mom_covs_c0_3_6r0_3_80micron_hardcase[:,1,1]),
         color='orange', label='C0-3-6R0-3, $80\mu m$')
plt.xlim(0., 3600.)
plt.ylim(0., 2.2)
plt.xlabel('Time (s)')
plt.ylabel('Cloud mass (g/kg)')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7, 4))
# 1.e3 to get mm^6/cm^3
m6_scale = (1.e3 * const.std_diameter)**6 * m0_scale
plt.plot(t_eval, m6_scale * moms_c0_3r0_3_80micron_hardcase[:,2],
         'k', label='Total $M_6$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3r0_3_80micron_hardcase[:,2,2]),
         color='b', label='C0-3R0-3, $80\mu m$')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_c0_3_6r0_3_80micron_hardcase[:,2,2]),
         color='orange', label='C0-3-6R0-3, $80\mu m$')
plt.xlim(0., 3600.)
plt.ylim(0., 1.1)
plt.xlabel('Time (s)')
plt.ylabel('$M_6$ ($mm^6cm^{-3}$)')
plt.legend()

# Plots of 1-category uncertainty (ellipsoids)

In [ ]:
wvs_ell = np.zeros((4, grid.num_bins))
wvs_ell[0,:] = grid.moment_weight_vector(0)
wvs_ell[1,:] = grid.moment_weight_vector(3)
wvs_ell[2,:] = grid.moment_weight_vector(6)
wvs_ell[3,:] = grid.moment_weight_vector(9)

In [ ]:
moms_ell, mom_covs_ell = \
    exp_u0_3_6_9.get_moments_and_covariances(wvs_ell)
# Convert to amount per cm^-3.
m0_scale = 1.e-6 * const.rho_air * const.mass_conc_scale / const.std_mass
scales = [
    m0_scale,
    1.e3 * const.mass_conc_scale,
    (1.e3 * const.std_diameter)**6 * m0_scale,
    (1.e3 * const.std_diameter)**9 * m0_scale,
]
scale_mat = np.diag(scales)
moms_ell = moms_ell @ scale_mat
mom_covs_ell = scale_mat @ mom_covs_ell @ scale_mat

In [ ]:
# Points to graph ellipse at.
pts = (0, 23, 34, 45)

In [ ]:
lengths_06 = []
angles_06 = []
lengths_09 = []
angles_09 = []
lengths_69 = []
angles_69 = []
lengths_069 = []
rot_069 = []
for pt in pts:
    pt_cov = mom_covs_ell[pt,:,:]
    eigval, eigvec = la.eigh(pt_cov)
    cov_06 = pt_cov[(0,2),:][:,(0,2)]
    eigval_06, eigvec_06 = la.eigh(cov_06)
    lengths_06.append(np.sqrt(eigval_06))
    angles_06.append((180. / np.pi)*np.arctan2(eigvec_06[0,1], eigvec_06[0,0]))
    cov_09 = pt_cov[(0,3),:][:,(0,3)]
    eigval_09, eigvec_09 = la.eigh(cov_09)
    lengths_09.append(np.sqrt(eigval_09))
    angles_09.append((180. / np.pi)*np.arctan2(eigvec_09[0,1], eigvec_09[0,0]))
    cov_69 = pt_cov[(2,3),:][:,(2,3)]
    eigval_69, eigvec_69 = la.eigh(cov_69)
    lengths_69.append(np.sqrt(eigval_69))
    angles_69.append((180. / np.pi)*np.arctan2(eigvec_69[0,1], eigvec_69[0,0]))
    cov_069 = pt_cov[(0,2,3),:][:,(0,2,3)]
    eigval_069, eigvec_069 = la.eigh(cov_069)
    lengths_069.append(np.sqrt(eigval_069))
    rot = np.zeros_like(eigvec_069)
    for i in range(3):
        rot[:,i] = eigvec_069[:,i] / la.norm(eigvec_069[:,i])
    rot_069.append(rot)

In [ ]:
rot_069

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(moms_ell[:,0], moms_ell[:,2], 'k')
for i, pt in enumerate(pts):
    ellipse = matplotlib.patches.Ellipse((moms_ell[pt,0], moms_ell[pt,2]),
                                         lengths_06[i][0], lengths_06[i][1],
                                         angle=angles_06[i], edgecolor='b',
                                         facecolor='lightgrey', lw=2)
    plt.gca().add_patch(ellipse)
plt.xlabel('$M_0$ ($cm^{-3}$)')
plt.ylabel('$M_6$ ($mm^6cm^{-3}$)')

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(moms_ell[:,0], moms_ell[:,3], 'k')
for i, pt in enumerate(pts):
    ellipse = matplotlib.patches.Ellipse((moms_ell[pt,0], moms_ell[pt,3]),
                                         lengths_09[i][0], lengths_09[i][1],
                                         angle=angles_09[i], edgecolor='b',
                                         facecolor='lightgrey', lw=2)
    plt.gca().add_patch(ellipse)
plt.xlabel('$M_0$ ($cm^{-3}$)')
plt.ylabel('$M_9$ ($mm^9cm^{-3}$)')

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(moms_ell[:,2], moms_ell[:,3], 'k')
for i, pt in enumerate(pts):
    ellipse = matplotlib.patches.Ellipse((moms_ell[pt,2], moms_ell[pt,3]),
                                         lengths_69[i][0], lengths_69[i][1],
                                         angle=angles_69[i], edgecolor='b',
                                         facecolor='lightgrey', lw=2)
    plt.gca().add_patch(ellipse)
plt.xlabel('$M_6$ ($mm^6cm^{-3}$)')
plt.ylabel('$M_9$ ($mm^9cm^{-3}$)')

In [ ]:
ax = plt.figure(figsize=(6,6)).add_subplot(projection='3d')
# Plot trajectory in space.
ax.plot(moms_ell[:,0], moms_ell[:,2], moms_ell[:,3], 'k')
# Locations of shadow planes.
x_low = -20.
y_low = -0.05
z_low = -10.
# Plot "shadow" curves on the box boundaries.
plt.plot(moms_ell[:,0], moms_ell[:,2], z_low, 'k', alpha=0.5)
plt.plot(moms_ell[:,0], y_low*np.ones_like(moms_ell[:,2]),
         moms_ell[:,3], 'k', alpha=0.5)
plt.plot(x_low*np.ones_like(moms_ell[:,0]), moms_ell[:,2],
         moms_ell[:,3], 'k', alpha=0.5)
# Coordinates of points on a sphere.
num_phi = 11
num_theta = 21
phi = np.linspace(0., 2.*np.pi, num_phi)
theta = np.linspace(-np.pi, np.pi, num_theta)
x = np.outer(np.cos(phi), np.sin(theta))
y = np.outer(np.sin(phi), np.sin(theta))
z = np.outer(np.ones((num_phi,)), np.cos(theta))
colors = ('b', 'g', 'r', 'grey')
for i, pt in enumerate(pts):
    # Center of ellipse.
    center = moms_ell[pt,(0,2,3)]
    # Stretch sphere into ellipse.
    pt_coord = np.zeros((3, x.shape[0], x.shape[1]))
    pt_coord[0,:,:] = 0.5 * lengths_069[i][0] * x
    pt_coord[1,:,:] = 0.5 * lengths_069[i][1] * y
    pt_coord[2,:,:] = 0.5 * lengths_069[i][2] * z
    # Rotate ellipse to proper orientation and translate.
    for j in range(num_phi):
        for k in range(num_theta):
            pt_coord[:,j,k] = rot_069[i] @ pt_coord[:,j,k] + center
    ax.plot_wireframe(pt_coord[0,:,:], pt_coord[1,:,:], pt_coord[2,:,:],
                      color='b', alpha=0.2)
    # Shadow ellipses
    ellipse = matplotlib.patches.Ellipse((moms_ell[pt,0], moms_ell[pt,2]),
                                         lengths_06[i][0], lengths_06[i][1],
                                         angle=angles_06[i], edgecolor='b',
                                         facecolor='lightgrey',
                                         lw=2, alpha=0.5)
    ax.add_patch(ellipse)
    art3d.pathpatch_2d_to_3d(ellipse, z=z_low, zdir="z")
    ellipse = matplotlib.patches.Ellipse((moms_ell[pt,0], moms_ell[pt,3]),
                                         lengths_09[i][0], lengths_09[i][1],
                                         angle=angles_09[i], edgecolor='b',
                                         facecolor='lightgrey',
                                         lw=2, alpha=0.5)
    ax.add_patch(ellipse)
    art3d.pathpatch_2d_to_3d(ellipse, z=y_low, zdir="y")
    ellipse = matplotlib.patches.Ellipse((moms_ell[pt,2], moms_ell[pt,3]),
                                         lengths_69[i][0], lengths_69[i][1],
                                         angle=angles_69[i], edgecolor='b',
                                         facecolor='lightgrey',
                                         lw=2, alpha=0.5)
    ax.add_patch(ellipse)
    art3d.pathpatch_2d_to_3d(ellipse, z=x_low, zdir="x")
plt.xlabel('$M_0$ ($cm^{-3}$)')
plt.ylabel('$M_6$ ($mm^6cm^{-3}$)')
ax.set_zlabel('$M_9$ ($mm^9cm^{-3}$)')
#ax.view_init(elev=0., azim=-90., roll=0)
ax.view_init(elev=20., azim=60., roll=0)
ax.dist = 12.